In [0]:
# bibliotecas usadas
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 

In [0]:
# para gerar o grafico no browser
%matplotlib inline

In [4]:
# local dos dados e modifica o diretorio
!git clone https://github.com/ucfilho/ANN
%cd ANN
!ls

Cloning into 'ANN'...
remote: Counting objects: 57, done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 57 (delta 0), reused 0 (delta 0), pack-reused 54
Unpacking objects: 100% (57/57), done.
/content/ANN
ann_aula_ago_14_2018_leitura_git.ipynb
ANN_aula_Scikit_learn _ago_29_2018.ipynb
ANN_Hydrofoil_ago_29_2018.txt.ipynb
ANN_Hydrofoil_Clean_02_set_12_2018.ipynb
ANN_Hydrofoil_Neuronicos_02__set_12_2018.ipynb
ANN_Hydrofoil_Neuronio_set_17_2018.ipynb
ANN_Hydrofoil_set_05_2018.ipynb
ANN_Hydrofoil_set_12_2018.ipynb
Aprendendo_Um_Pouco_mais_sobre_Python_set_05_2018.ipynb
arvore.jpg
Aula_003_Python_audio_e_imagem_ago_29_2018.ipynb
aula_04_rede_neural_set_05_2018.pdf
BBC_007041008.wav
Constroi_ANN_set_19_2018.ipynb
Criando_Gethub_Ver_Youtube.txt
FAULT_DIAGNOSIS_maio_25_2017.txt
Hydrofoil_ago_29_2018.csv
john_mccarthy.png
Paper_Carvao_China_abril_12_2016.csv
Paper_Carvao_China_mar_31_2016.txt
README.md
simbolos.pdf
Video_mp4_Python_Notebook.ipynb


In [0]:
# leitura dos dados
DAT=pd.read_csv("Hydrofoil_ago_29_2018.csv")
#DAT=pd.read_csv("Hydrofoil_ago_29_2018.csv",index_col="emit")

In [0]:
# separa as colunas de interesse
Index=[2,3,4,5,6,7]
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [0]:
# separa os dados em normatizados e originais
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)

In [10]:
# separa os dados em treino e teste
train=[]
test=[]
train=DAT.sample(frac=0.8,random_state=None).copy() # poderia ser frac=0.5? sim ou nao?
test=DAT.drop(train.index).copy()
print(train.head())

    Hydrofoil    Froude  Resistence   Sinkage      Trim     Power
3   -1.000000 -0.861711   -0.966146 -0.916618 -0.456079 -0.997659
15  -1.000000  0.728608    0.626042  0.702954  0.911268  0.405395
6   -1.000000 -0.654278   -0.644271 -1.000000 -0.179187 -0.938508
56   1.000000  0.728608    0.458333  0.883962  0.180864  0.260444
25  -0.333333  0.382887    0.281250  0.515567  0.481548 -0.114088


In [0]:
# obtem as variaveis independentes (treino e teste)
x_train=train.iloc[:,[0,1]]
x_test=test.iloc[:,[0,1]]
X_OLD=DAT_OLD.iloc[:,[0,1]]

In [0]:
# obtem as variaveis dependentes (treino e teste)
Index=[2,3,4,5]
y_train=train.iloc[:,Index]
y_test=test.iloc[:,Index]
Y_OLD=DAT_OLD.iloc[:,Index]

In [0]:
def rede(i,j):
    clf = MLPRegressor(solver='lbfgs',activation='tanh',alpha=1e-5,
                       hidden_layer_sizes=(i, j),random_state=1)
    return clf

In [14]:
N=10
M=10
test=Original(y_test,Y_OLD)
resist_obs_test=[]
resist_obs_test=test['Resistence'].copy()
Melhor=1e99 #toda vez que fizer o teste e o valor da iteracao der menor que o erro. r^2 nao costuma ser o melhor para calcular o erro. MSE=1/n(yexp-ycal)^2 
ir=0
jr=0
for i in range(1,N): #numero de neuronios da primeira camada oculta
    for j in range(1,M):
        col_names=list(y_train)
        clf=rede(i,j)
        clf.fit(x_train, y_train)
        y_calc_train=[]
        y_calc_test=[]
        y_calc_train=clf.predict(x_train)
        y_calc_test=clf.predict(x_test)
        y_calc_train=pd.DataFrame(y_calc_train)
        y_calc_test=pd.DataFrame(y_calc_test)
        y_calc_train.columns = col_names
        y_calc_test.columns = col_names
        y_calc_train=Original(y_calc_train,Y_OLD)
        y_calc_test=Original(y_calc_test,Y_OLD)
        resist_calc_test=[]
        resist_calc_test=y_calc_test['Resistence'].copy()
        mse=mean_squared_error(resist_obs_test,resist_calc_test)
        R2=r2_score(resist_obs_test,resist_calc_test)
        print("Neuronios=",i,j,"MSE teste=",mse,"R^2 teste=",R2)
        if(mse<Melhor):
            Melhor=mse.copy()
            ix=i
            jx=j
            R2r=R2
print("\n Melhor resposta:")
print("Neuronios=",ix,jx,"MSE teste=",Melhor,"R^2 teste=",R2r)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Neuronios= 1 1 MSE teste= 602.2210240034559 R^2 teste= 0.9017584808795475
Neuronios= 1 2 MSE teste= 600.0756551565088 R^2 teste= 0.9021084591868418
Neuronios= 1 3 MSE teste= 280.7718241298132 R^2 teste= 0.9541971312370272
Neuronios= 1 4 MSE teste= 602.6052777885842 R^2 teste= 0.9016957967916891
Neuronios= 1 5 MSE teste= 316.03086499160764 R^2 teste= 0.9484452534397936
Neuronios= 1 6 MSE teste= 596.7302022138371 R^2 teste= 0.9026542096109124
Neuronios= 1 7 MSE teste= 318.40914656297144 R^2 teste= 0.9480572796143139
Neuronios= 1 8 MSE teste= 433.4157844853418 R^2 teste= 0.9292960169414838
Neuronios= 1 9 MSE teste= 335.4268923480596 R^2 teste= 0.9452811407362345
Neuronios= 2 1 MSE teste= 610.8249461340148 R^2 teste= 0.9003549058683646
Neuronios= 2 2 MSE teste= 648.7374290787997 R^2 teste= 0.8941701667615105
Neuronios= 2 3 MSE teste= 233.06862152480517 R^2 teste= 0.9619790500077672
Neuronios= 2 4 MSE teste= 131.35603489512428 R^2 teste= 0.9785716275264709
Neuronios= 2 5 MSE teste= 675.7084